In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('data/train.csv')



/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
df['amount']

0         0.0
1         0.0
2         0.0
3         1.0
4         0.0
5         0.0
6         0.0
7         0.0
8         0.0
9         0.0
10        0.0
11        0.0
12        0.0
13        0.0
14        0.0
15        0.0
16        0.0
17        0.0
18        0.0
19        0.0
20        0.0
21        0.0
22        1.0
23        0.0
24        0.0
25        0.0
26        0.0
27        0.0
28        1.0
29        0.0
         ... 
182160    0.0
182161    0.0
182162    0.0
182163    0.0
182164    0.0
182165    0.0
182166    0.0
182167    1.0
182168    0.0
182169    0.0
182170    0.0
182171    0.0
182172    0.0
182173    0.0
182174    0.0
182175    0.0
182176    0.0
182177    0.0
182178    0.0
182179    0.0
182180    0.0
182181    0.0
182182    0.0
182183    0.0
182184    0.0
182185    0.0
182186    0.0
182187    0.0
182188    0.0
182189    0.0
Name: amount, Length: 182190, dtype: float64

In [3]:
df[df == ' '] = np.nan
df['amount'] = df['amount'].fillna(0)

In [4]:
df['amount'][df['amount'] > 0] = 1

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [7]:
df = df.dropna(axis=1, thresh=df.shape[0]//2 + 1)
for col in df.columns:
    df[col] = df[col].fillna(df[col].value_counts().keys()[0])
    if df[col].dtype == 'object':
        df[col] = pd.factorize(df[col])[0]

In [8]:
df.columns

Index([u'date', u'source', u'title', u'state', u'zip', u'dob', u'noexch',
       u'mdmaud', u'domain', u'cluster',
       ...
       u'amount', u'hphone_d', u'rfa_2r', u'rfa_2f', u'rfa_2a', u'mdmaud_r',
       u'mdmaud_f', u'mdmaud_a', u'cluster2', u'geocode2'],
      dtype='object', length=382)

In [9]:
df.shape

(182190, 382)

In [10]:
m = df.shape[0] # row
n = df.shape[1] # col

from sklearn.cluster import KMeans
random_state = 170

print df.shape
y = KMeans(n_clusters=2, random_state=random_state).fit_predict(df)


(182190, 382)


In [11]:
output = open('output.txt', 'w+')
print >>output, y
print y.sum()

91112


In [12]:
sum(df['amount'])

9249.0

In [33]:
df.drop(columns=['responded'], inplace = True)
train_label = df['amount']
df2 = pd.DataFrame()
for attri in df.columns:
    if attri != 'amount':
        df2[attri] = df[attri]
train_data = df2
print train_label.shape
print train_data.shape

(182190,)
(182190, 380)


In [34]:
from sklearn.model_selection import ShuffleSplit
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(np.array(train_data), np.array(train_label))

cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
scores = cross_val_score(clf, train_data, train_label, cv = cv)
print scores

[0.95098524 0.95150667 0.94996981 0.94994237 0.95029914]


In [36]:
test = pd.read_csv('data/test.csv')

test.shape
#clf.predict()

(9589, 480)

In [37]:
temp = list(test.columns)
for col in temp:
    if col not in df.columns:
        #print(col)
        test = test.drop(columns=[col])

In [38]:
test.shape

(9589, 380)

In [44]:
for col in test.columns:
    test[col] = test[col].fillna(df[col].value_counts().keys()[0])
    if test[col].dtype == 'object':
        test[col] = pd.factorize(test[col])[0]

In [45]:
print test

      date  source  title  state   zip   dob  noexch  mdmaud  domain  cluster  \
0     9301       0      1      0     0  2001       0       0       0        0   
1     9101       1   1002      1     1     0       0       0       1        1   
2     8601       2      0      2     2  4305       0       0       2        2   
3     8601       3      0      3     3     0       0       0       0        3   
4     9501       4      0      4     4  4201       0       0       2        2   
5     9401       5      0      5     5  3501       0       0       1        4   
6     8601       6      0      6     6  2401       0       0       3        5   
7     9401       7      2      7     7  2401       0       0       4        6   
8     9001       8      1      1     8  2712       0       0       5        7   
9     8601       3      2      8     9     0       0       0       2        8   
10    8801       9      0      1    10     0       0       0       6        9   
11    9001       1     28   

In [46]:
predict = clf.predict(test)
print predict

[0. 0. 0. ... 0. 0. 0.]


In [47]:
predict.sum()

0.0